## AEP calculation

$$AEP = \sum_{wt=0}^{N}\sum_{wd_{ref}=0}^{360}\sum_{ws = cut{\text -}in}^{cut{\text -}out} Power \cdot Weight$$

$$Power = PowerCurve_{wt}\left(LocalWind,\theta_{yaw}\right)\cdot \frac{\rho}{\rho_{ref}}$$

$$\begin{align}LocalWind = WS \cdot Speedup - WakeDeficits\end{align}$$

$$  Weight=\begin{cases}
    P, & \text{if $P$ is provided}.\\
    f \cdot Weibull(A,k,ws), & \text{otherwise}.
  \end{cases}$$

Where 

- $wt$: wind turbine
- $wd$: reference wind direction
- $ws$: reference wind speed
- $\theta_{yaw}$: yaw missalignment of the wind turbine
- $PowerCurve_{wt}$: the wind-speed and yaw-misalignment dependent power curve of the wind turbine
- $WakeDeficits$: Deficits from other wind turbines

Required data variables:

- $P$: Probability of flow case

or

- $f$: Sector probability
- $A$: Weibull scale parameter
- $k$: Weibull shape parameter

Optional data variables:

- $WS$: Wind speed, defaults to reference wind speed, $ws$
- $TI$: Turbulence intensity
- $Speedup$: Speed up factor, defaults to 1
- $Turning$: change in wind direction
- $InflowAngle$: angle/slope of inflow
- $\rho$: Air density, defaults to reference density, $\rho_{ref}$

These required and optional data varaibles may be constant or dependent on any of
- $ws$ 
- $wd$
- $position$ in terms of
  - gridded 2D position, (x,y)
  - gridded 3D position, (x,y,z)
  - wt position

In [24]:
import xarray as xr
import numpy as np
ds = xr.Dataset(
        data_vars={'TI': (['x', 'y', 'z','ws','wd'], np.random.random((300,300,5,23,12)))},
        coords={'wd': np.linspace(0, 360, 12, endpoint=False),
                'x': np.linspace(-20000,20000, 300),
                'y': np.linspace(-20000,20000, 300),
                'z': [30,60,90,120,150],
                'ws': np.arange(3,26),
                'wd': np.arange(12)*30
                
               })

ds

<xarray.Dataset>
Dimensions:  (wd: 12, ws: 23, x: 300, y: 300, z: 5)
Coordinates:
  * wd       (wd) int32 0 30 60 90 120 150 180 210 240 270 300 330
  * x        (x) float64 -2e+04 -1.987e+04 -1.973e+04 ... 1.987e+04 2e+04
  * y        (y) float64 -2e+04 -1.987e+04 -1.973e+04 ... 1.987e+04 2e+04
  * z        (z) int32 30 60 90 120 150
  * ws       (ws) int32 3 4 5 6 7 8 9 10 11 12 ... 16 17 18 19 20 21 22 23 24 25
Data variables:
    TI       (x, y, z, ws, wd) float64 0.4522 0.7198 0.03985 ... 0.8615 0.6197

In [18]:
print(yaml.dump(ds.to_dict(data=False)))

attrs: {}
coords:
  wd:
    attrs: {}
    dims: !!python/tuple
    - wd
    dtype: float64
    shape: !!python/tuple
    - 4
  x:
    attrs: {}
    dims: !!python/tuple
    - x
    dtype: float64
    shape: !!python/tuple
    - 3
  y:
    attrs: {}
    dims: !!python/tuple
    - y
    dtype: float64
    shape: !!python/tuple
    - 3
data_vars:
  TI:
    attrs: {}
    dims: !!python/tuple
    - x
    - y
    - wd
    dtype: float64
    shape: !!python/tuple
    - 3
    - 3
    - 4
dims:
  wd: 4
  x: 3
  y: 3



In [16]:
from examples.example_data.make_example_data import xr2yml
print(xr2yml(ds))

coords:
  wd:
    data:
    - 0.0
    - 90.0
    - 180.0
    - 270.0
    dims:
    - wd
  x:
    data:
    - -2000.0
    - 0.0
    - 2000.0
    dims:
    - x
  y:
    data:
    - -2000.0
    - 0.0
    - 2000.0
    dims:
    - y
data_vars:
  TI:
    data:
    - - - 0.35986112762995404
        - 0.2502664560195994
        - 0.12869611037695716
        - 0.6881782007898395
      - - 0.7399800121006861
        - 0.532894408640326
        - 0.08135059873291284
        - 0.8327840678535788
      - - 0.061908471364790496
        - 0.031035289934175703
        - 0.11182911765884485
        - 0.06181945915029197
    - - - 0.9704429283226162
        - 0.9243111168297061
        - 0.2731016800961287
        - 0.8903470701846943
      - - 0.07808850983441074
        - 0.6987806914385957
        - 0.5190023325212555
        - 0.5308198291470925
      - - 0.46483717515716416
        - 0.0022911352126614437
        - 0.4239872343277624
        - 0.0047723667403831715
    - - - 0.576114505057546
     

In [20]:
    f = [.025, .024, .029, .036, .063, .065, .100, .122, .063, .038, .039, .083, .213, .046, .032, .022]
    WS = 9.8
    ds = xr.Dataset(
        data_vars={'WS': WS, 'P': ('wd', f)},
        coords={'wd': np.linspace(0, 360, len(f), endpoint=False)})

In [21]:
ds

<xarray.Dataset>
Dimensions:  (wd: 16)
Coordinates:
  * wd       (wd) float64 0.0 22.5 45.0 67.5 90.0 ... 270.0 292.5 315.0 337.5
Data variables:
    WS       float64 9.8
    P        (wd) float64 0.025 0.024 0.029 0.036 ... 0.213 0.046 0.032 0.022

In [22]:
from examples.example_data.make_example_data import xr2yml
print(xr2yml(ds))

coords:
  wd:
    data:
    - 0.0
    - 22.5
    - 45.0
    - 67.5
    - 90.0
    - 112.5
    - 135.0
    - 157.5
    - 180.0
    - 202.5
    - 225.0
    - 247.5
    - 270.0
    - 292.5
    - 315.0
    - 337.5
    dims:
    - wd
data_vars:
  P:
    data:
    - 0.025
    - 0.024
    - 0.029
    - 0.036
    - 0.063
    - 0.065
    - 0.1
    - 0.122
    - 0.063
    - 0.038
    - 0.039
    - 0.083
    - 0.213
    - 0.046
    - 0.032
    - 0.022
    dims:
    - wd
  WS:
    data: 9.8
    dims: []
dims:
  wd: 16

